### This project includes three parts:
1. Data Analysis & Distribution (EDA)
2. Time Series Analysis
3. NLP Processing & Text Analysis  
  
### This notebook is the first part: Data Analysis & Distribution (EDA)

In [6]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

# Read Data

In [7]:
clients = pd.read_csv("/Users/sqtang37/Desktop/Project/DataFest 2023/data/clients.csv")
questions = pd.read_csv("/Users/sqtang37/Desktop/Project/DataFest 2023/data/questions.csv")
attorneys = pd.read_csv("/Users/sqtang37/Desktop/Project/DataFest 2023/data/attorneys.csv")
questions = pd.read_csv("/Users/sqtang37/Desktop/Project/DataFest 2023/data/questions.csv")

In [8]:
qp = []

with open('/Users/sqtang37/Desktop/Project/DataFest 2023/data/questionposts copy.txt', 'r') as file:
    for line in file:
        qp.append(line)
        
Qposts = pd.DataFrame([line.split(',', 3) for line in qp], columns=['Id', 'StateAbbr', 'QuestionUno', 'PostText'])

Qposts[['PostText', 'CreatedUtc']] = Qposts['PostText'].str.rsplit(',', 1, expand=True)

In [9]:
Qposts['PostText'] = Qposts['PostText'].str.replace(r'^"|"$', '', regex=True)
Qposts['CreatedUtc'] = Qposts['CreatedUtc'].str.replace(r'"', '', regex=True).str.replace(r'\n', '', regex=True)

Qposts = Qposts.iloc[1:]

In [10]:
attorneys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11544 entries, 0 to 11543
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id           11544 non-null  int64 
 1   StateAbbr    11544 non-null  object
 2   AttorneyUno  11544 non-null  object
 3   City         11529 non-null  object
 4   County       11539 non-null  object
 5   StateName    11544 non-null  object
 6   PostalCode   11544 non-null  int64 
 7   CreatedUtc   11544 non-null  object
dtypes: int64(2), object(6)
memory usage: 721.6+ KB


In [11]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331426 entries, 0 to 331425
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Id                  331426 non-null  int64  
 1   StateAbbr           331426 non-null  object 
 2   ClientUno           331426 non-null  object 
 3   County              331180 non-null  object 
 4   StateName           331426 non-null  object 
 5   PostalCode          314157 non-null  float64
 6   EthnicIdentity      278298 non-null  object 
 7   Age                 279870 non-null  float64
 8   Gender              280227 non-null  object 
 9   MaritalStatus       279548 non-null  object 
 10  Veteran             269757 non-null  object 
 11  Imprisoned          280602 non-null  object 
 12  NumberInHousehold   280602 non-null  float64
 13  AnnualIncome        280565 non-null  float64
 14  AllowedIncome       280602 non-null  float64
 15  CheckingBalance     212020 non-nul

In [12]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202879 entries, 0 to 202878
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Id                   202879 non-null  int64 
 1   StateAbbr            202879 non-null  object
 2   QuestionUno          202879 non-null  object
 3   CategoryUno          202879 non-null  object
 4   Category             202879 non-null  object
 5   SubcategoryUno       202879 non-null  object
 6   Subcategory          202879 non-null  object
 7   AskedByClientUno     202841 non-null  object
 8   AskedOnUtc           202879 non-null  object
 9   TakenByAttorneyUno   144005 non-null  object
 10  TakenOnUtc           145651 non-null  object
 11  ClosedByAttorneyUno  137787 non-null  object
 12  ClosedOnUtc          199085 non-null  object
 13  LegalDeadline        43944 non-null   object
dtypes: int64(1), object(13)
memory usage: 21.7+ MB


In [13]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202879 entries, 0 to 202878
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Id                   202879 non-null  int64 
 1   StateAbbr            202879 non-null  object
 2   QuestionUno          202879 non-null  object
 3   CategoryUno          202879 non-null  object
 4   Category             202879 non-null  object
 5   SubcategoryUno       202879 non-null  object
 6   Subcategory          202879 non-null  object
 7   AskedByClientUno     202841 non-null  object
 8   AskedOnUtc           202879 non-null  object
 9   TakenByAttorneyUno   144005 non-null  object
 10  TakenOnUtc           145651 non-null  object
 11  ClosedByAttorneyUno  137787 non-null  object
 12  ClosedOnUtc          199085 non-null  object
 13  LegalDeadline        43944 non-null   object
dtypes: int64(1), object(13)
memory usage: 21.7+ MB


In [14]:
Qposts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406188 entries, 1 to 406188
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           406188 non-null  object
 1   StateAbbr    405593 non-null  object
 2   QuestionUno  405589 non-null  object
 3   PostText     405580 non-null  object
 4   CreatedUtc   405570 non-null  object
dtypes: object(5)
memory usage: 15.5+ MB


---

# Data Analysis and Distribution
1. Number of clients vs. attorneys in the same state
2. Active clients & attorneys in each state
3. Response rate in each state
4. Do attorneys reply to clients from the same state?

### Determining the caseload for an attorney based on the number of clients within the same state

In [15]:
attorney_grouped = attorneys.groupby("StateAbbr").size()
client_grouped = clients.groupby("StateAbbr").size()
attorney_client = pd.concat([attorney_grouped, client_grouped], axis=1, keys=["Attorneys", "Clients"]).fillna(0)

attorney_client["Ratio"] = attorney_client["Clients"]/attorney_client["Attorneys"]

In [16]:
attorney_client = attorney_client.sort_values(by='Ratio', ascending=True)
attorney_client[:5]
# More attorneys may be needed for these states

,Attorneys,Clients,Ratio
StateAbbr,,,
VI,3,0.0,0.000000
ND,1,0.0,0.000000
ID,2,1.0,0.500000
AL,138,1538.0,11.144928
MI,77,881.0,11.441558


In [17]:
attorney_client = attorney_client.sort_values(by='Attorneys', ascending=True)
attorney_client[:5]
# More attorneys may be needed for these states

,Attorneys,Clients,Ratio
StateAbbr,,,
ND,1,0.0,0.000000
ID,2,1.0,0.500000
VI,3,0.0,0.000000
SD,26,1503.0,57.807692
KS,30,875.0,29.166667


In [18]:
attorney_client = attorney_client.sort_values(by='Clients', ascending=True)
attorney_client[:5]
# More clients may the association develop for these states

,Attorneys,Clients,Ratio
StateAbbr,,,
ND,1,0.0,0.000000
VI,3,0.0,0.000000
ID,2,1.0,0.500000
KS,30,875.0,29.166667
MI,77,881.0,11.441558


### Active clients & attorneys in each state

In [19]:
active_clients = questions["AskedByClientUno"].groupby(questions["StateAbbr"]).nunique()
active_clients_rate = active_clients / client_grouped

active_attorneys = questions["TakenByAttorneyUno"].groupby(questions["StateAbbr"]).nunique()
active_attorneys_rate = active_attorneys / attorney_grouped

In [20]:
active_rate = pd.concat([active_clients_rate, active_attorneys_rate], axis=1, 
                        keys=["active_clients_rate", "active_attorneys_rate"]).fillna(0)

active_rate = active_rate.sort_values(by=['active_attorneys_rate', "active_clients_rate"], ascending=[True, True])
active_rate[:5]
# More attorneys may be needed for these states

,active_clients_rate,active_attorneys_rate
StateAbbr,,
ND,0.000000,0.000000
VI,0.000000,0.000000
ID,1.000000,0.000000
PA,0.437068,0.183673
US,0.273245,0.247191


### Response rate in each state

In [21]:
questions_taken = questions.groupby("StateAbbr")["TakenByAttorneyUno"].count()
total_questions = questions.groupby("StateAbbr")["TakenByAttorneyUno"].size()
response_rate = questions_taken / total_questions
response_rate.sort_values()[:5]

StateAbbr
ID    0.000000
PA    0.181818
GA    0.204154
KS    0.242972
AZ    0.246013
Name: TakenByAttorneyUno, dtype: float64

### Do attorneys reply to clients from the same state?

In [22]:
merged_data = questions[['AskedByClientUno', 'TakenByAttorneyUno']] \
                        .merge(clients[['StateAbbr', 'ClientUno']], how='left', left_on='AskedByClientUno', right_on='ClientUno') \
                        .merge(attorneys[['StateAbbr', 'AttorneyUno']], how='left', left_on='TakenByAttorneyUno', right_on='AttorneyUno')

merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202879 entries, 0 to 202878
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   AskedByClientUno    202841 non-null  object
 1   TakenByAttorneyUno  144005 non-null  object
 2   StateAbbr_x         202841 non-null  object
 3   ClientUno           202841 non-null  object
 4   StateAbbr_y         144005 non-null  object
 5   AttorneyUno         144005 non-null  object
dtypes: object(6)
memory usage: 10.8+ MB


In [23]:
merged_data['responded_by_same_state'] = merged_data['StateAbbr_x'] == merged_data['StateAbbr_y']
responded_by_same = merged_data.groupby('StateAbbr_x')['responded_by_same_state'].sum()
merged_data.drop('responded_by_same_state', axis=1, inplace=True)

merged_data['responded_by_diff_state'] = merged_data['StateAbbr_x'] != merged_data['StateAbbr_y']
responded_by_diff = merged_data.groupby('StateAbbr_x')['responded_by_diff_state'].sum()
merged_data.drop('responded_by_diff_state', axis=1, inplace=True)

response_state = pd.concat([responded_by_same, responded_by_diff], axis=1, 
                        keys=["responded_by_same", "responded_by_diff"])

response_state = response_state.sort_values(by='responded_by_same', ascending=True)
response_state[:5]

,responded_by_same,responded_by_diff
StateAbbr_x,,
ID,0,1
US,117,188
KS,121,377
PA,126,567
MI,340,172


---

### For the other two parts of the project, please refer to the other two ipynb
2. Time Series Analysis
3. NLP Processing & Text Analysis